# 导入库函数

In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Parameter
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
from torch_geometric.datasets import Planetoid
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# evaluating indicator

In [2]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    ap = average_precision_score(true_edges, predict_edges)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    return ap, auc

# model

In [3]:
"""  Given the cluster centroids, update the node representation  """
class reconstruction_graph(nn.Module):
    def __init__(self, NE, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, CC):
        # Calculate the attention score (in nodes) for each node to each cluster
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # Distance from the node to the center of mass
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power    
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
        prod = torch.mm(soft_assignments, soft_assignments.t())       #  numerator
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # Calculate the distance between nodes: smaller values favor edge formation
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, CC, edge_train, edge_test):
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        ap, auc = predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach(), ap, auc
    
"""  Type-centroid update aimed at maximizing the prediction effect   """
class ClusteringLayer(nn.Module):
    """  Given the node representation, update the cluster centroids  """
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, NE):
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2) 
        numerator = 1.0 / (1.0 + (norm_squared / DEC))
        power = float(self.alpha + 1) / 2   
        numerator = numerator**power   
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() 
        
        prod = torch.mm(soft_assignments, soft_assignments.t())
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(NE, CC, g, edge_train, edge_test, num_epochs=5):        
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=LR, momentum=0.9)
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    ap, auc = predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach(), ap, auc

In [4]:
def get_cora_graph(filename_adj, nodes_number):
    datasets = Planetoid('./datasets', filename_adj)
    edges = datasets[0].edge_index
    
    #print("edges number ： ", edges.shape[1])
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[1]):
        graph_np[edges[0,i], edges[1,i]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # Only the upper-left corner matrix is taken
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # Only the upper-left corner matrix is taken
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*4,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [5]:
filename = "Cora"    # The node index starts at 0, sep=None
nodes_number = 2708

embedding_dim = 24
n_clusters = 175
alpha = 1
beta = 4.5
DEC = 4.5
LR = 0.4

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []

for i in range(10):
    
    train_edge, test_edge, train_mask, test_mask = get_cora_graph(filename, nodes_number)
    
    ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)       
    raw_nodes_embedding = ini_embedding.weight.detach()        # Initial node embedding

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids
    
    ap_first__ap = 0
    ap_first_auc = 0
    auc_first__ap = 0
    auc_first_auc = 0
    for module_epoch in range(300):
        print("######################### Module cycle ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
        raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
        if nodes_ap > ap_first__ap:
            ap_first__ap = nodes_ap
            ap_first_auc = nodes_auc
        if nodes_auc > auc_first_auc:
            auc_first_auc = nodes_auc
            auc_first__ap = nodes_ap
        raw_cluster_centers, cluster_ap, cluster_auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)
        if cluster_ap > ap_first__ap:
            ap_first__ap = cluster_ap
            ap_first_auc = cluster_auc
        if cluster_auc > auc_first_auc:
            auc_first_auc = cluster_auc
            auc_first__ap = cluster_ap
            
    all_ap_first_ap.append(ap_first__ap)
    all_ap_first_auc.append(ap_first_auc)
    all_auc_first_ap.append(auc_first__ap)
    all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
AP：  0.6169726905735808
AUC SCORE:  0.5996757346189164
AP：  0.635503680355747
AUC SCORE:  0.6178833792470156
######################### Module cycle ： 1 ##########################
AP：  0.6618303873149476
AUC SCORE:  0.6343979568411386
AP：  0.6661910440105503
AUC SCORE:  0.6370092975206612
######################### Module cycle ： 2 ##########################
AP：  0.6654135961323078
AUC SCORE:  0.6361340679522497
AP：  0.665886866527184
AUC SCORE:  0.6324071682736455
######################### Module cycle ： 3 ##########################
AP：  0.6713332934534716
AUC SCORE:  0.6360910238751147
AP：  0.6738292279725365
AUC SCORE:  0.6395991161616161
######################### Module cycle ： 4 ##########################
AP：  0.6867147552786452
AUC SCORE:  0.6559594524793388
AP：  0.6893410855649331
AUC SCORE:  0.6551380280073462
######################### Module cycle ： 5 ##########################
AP：  0.6901465844490411
AUC SCOR

AP：  0.8567248464023938
AUC SCORE:  0.820229998852158
######################### Module cycle ： 46 ##########################
AP：  0.8575150160938991
AUC SCORE:  0.8213599058769513
AP：  0.8577946803913366
AUC SCORE:  0.8218477387511478
######################### Module cycle ： 47 ##########################
AP：  0.8583084630016541
AUC SCORE:  0.8229310146923783
AP：  0.858640311937093
AUC SCORE:  0.823408086547291
######################### Module cycle ： 48 ##########################
AP：  0.8592180286314252
AUC SCORE:  0.8244626664370983
AP：  0.8595321300415912
AUC SCORE:  0.8248393021120293
######################### Module cycle ： 49 ##########################
AP：  0.8601686330841405
AUC SCORE:  0.8258149678604225
AP：  0.860503684284639
AUC SCORE:  0.8262238865932048
######################### Module cycle ： 50 ##########################
AP：  0.861014557268092
AUC SCORE:  0.8270166150137741
AP：  0.8613037953862747
AUC SCORE:  0.8275367309458219
######################### Module cycle ： 51 #

AP：  0.8911610653334997
AUC SCORE:  0.8643071338383838
AP：  0.8912602428435341
AUC SCORE:  0.8643752869605141
######################### Module cycle ： 92 ##########################
AP：  0.8913163314587348
AUC SCORE:  0.8645259412304866
AP：  0.8913695012879598
AUC SCORE:  0.8646263774104683
######################### Module cycle ： 93 ##########################
AP：  0.8914622719719514
AUC SCORE:  0.8648129017447199
AP：  0.8915760427422983
AUC SCORE:  0.8650173611111109
######################### Module cycle ： 94 ##########################
AP：  0.8915474893763838
AUC SCORE:  0.865063992194674
AP：  0.8914365280515382
AUC SCORE:  0.8649958390725437
######################### Module cycle ： 95 ##########################
AP：  0.8914996725016501
AUC SCORE:  0.8651644283746557
AP：  0.8915323154354752
AUC SCORE:  0.8651787764003673
######################### Module cycle ： 96 ##########################
AP：  0.8916520663399969
AUC SCORE:  0.8653079086317723
AP：  0.891813892059842
AUC SCORE:  0.8655

AP：  0.8951591506629701
AUC SCORE:  0.8736190025252526
######################### Module cycle ： 137 ##########################
AP：  0.8950923113630487
AUC SCORE:  0.8736799816345271
AP：  0.8950216964076851
AUC SCORE:  0.8737373737373737
######################### Module cycle ： 138 ##########################
AP：  0.8949482393639234
AUC SCORE:  0.8737911788337924
AP：  0.8949340212206544
AUC SCORE:  0.8738413969237833
######################### Module cycle ： 139 ##########################
AP：  0.8948279622173907
AUC SCORE:  0.8739167240587695
AP：  0.8947492866995621
AUC SCORE:  0.8739346590909091
######################### Module cycle ： 140 ##########################
AP：  0.8947208577866052
AUC SCORE:  0.8740279212580349
AP：  0.8946018392668966
AUC SCORE:  0.8739884641873278
######################### Module cycle ： 141 ##########################
AP：  0.894572271549108
AUC SCORE:  0.8740924873737375
AP：  0.894505586931928
AUC SCORE:  0.874060204315886
######################### Module cycle

AP：  0.8960256416885422
AUC SCORE:  0.8787699437557391
AP：  0.8960555025673863
AUC SCORE:  0.8787986398071626
######################### Module cycle ： 183 ##########################
AP：  0.8961104854508768
AUC SCORE:  0.878859618916437
AP：  0.8961598602157014
AUC SCORE:  0.8789421200642791
######################### Module cycle ： 184 ##########################
AP：  0.8961577671900757
AUC SCORE:  0.8789457070707071
AP：  0.8962476853060984
AUC SCORE:  0.8790569042699724
######################### Module cycle ： 185 ##########################
AP：  0.8962664444331483
AUC SCORE:  0.8790927743342516
AP：  0.896310863315805
AUC SCORE:  0.8791681014692377
######################### Module cycle ： 186 ##########################
AP：  0.8963261142877845
AUC SCORE:  0.8792147325528008
AP：  0.896403254505791
AUC SCORE:  0.8792757116620754
######################### Module cycle ： 187 ##########################
AP：  0.89645349534377
AUC SCORE:  0.8793653868227732
AP：  0.8964890256827052
AUC SCORE:  0.87

...

AP：  0.8546048286872876
AUC SCORE:  0.8208864210284665
AP：  0.8553851205844756
AUC SCORE:  0.822052198117539
######################### Module cycle ： 85 ##########################
AP：  0.8556139441450044
AUC SCORE:  0.8225651400367309
AP：  0.8562592367019614
AUC SCORE:  0.8236017848943985
######################### Module cycle ： 86 ##########################
AP：  0.8564565893857222
AUC SCORE:  0.8239891815886136
AP：  0.8570048645778601
AUC SCORE:  0.8248213670798898
######################### Module cycle ： 87 ##########################
AP：  0.8571643782316383
AUC SCORE:  0.8252302858126722
AP：  0.8576395729263699
AUC SCORE:  0.8259692091368227
######################### Module cycle ： 88 ##########################
AP：  0.8578561638334157
AUC SCORE:  0.8263314967860422
AP：  0.8583645824410191
AUC SCORE:  0.8271708562901745
######################### Module cycle ： 89 ##########################
AP：  0.8586060347958253
AUC SCORE:  0.8276658631772268
AP：  0.8589226985875298
AUC SCORE:  0.828

AP：  0.8684526314994286
AUC SCORE:  0.8485816976584021
######################### Module cycle ： 130 ##########################
AP：  0.8684926319217074
AUC SCORE:  0.8487000688705233
AP：  0.8685814393994973
AUC SCORE:  0.8488112660697889
######################### Module cycle ： 131 ##########################
AP：  0.8686883812022397
AUC SCORE:  0.848972681359045
AP：  0.8687457304417323
AUC SCORE:  0.8491233356290174
######################### Module cycle ： 132 ##########################
AP：  0.8688585623097878
AUC SCORE:  0.8492811639118457
AP：  0.8689492815498346
AUC SCORE:  0.8494569272268135
######################### Module cycle ： 133 ##########################
AP：  0.8691885094338825
AUC SCORE:  0.849733126721763
AP：  0.869314611538907
AUC SCORE:  0.8498586719467401
######################### Module cycle ： 134 ##########################
AP：  0.8694446744107845
AUC SCORE:  0.8500487832874196
AP：  0.8695225701055938
AUC SCORE:  0.8501241104224058
######################### Module cycle

AP：  0.8783946419633787
AUC SCORE:  0.8611326331496787
AP：  0.878530986681725
AUC SCORE:  0.8612653523875116
######################### Module cycle ： 176 ##########################
AP：  0.8785359470066982
AUC SCORE:  0.8612581783746556
AP：  0.8785880851771761
AUC SCORE:  0.8613406795224976
######################### Module cycle ： 177 ##########################
AP：  0.878644197408515
AUC SCORE:  0.8613837235996328
AP：  0.8786664296489284
AUC SCORE:  0.8613980716253444
######################### Module cycle ： 178 ##########################
AP：  0.8787322546128611
AUC SCORE:  0.8614733987603305
AP：  0.878766515710431
AUC SCORE:  0.8614985078053259
######################### Module cycle ： 179 ##########################
AP：  0.8788880593717526
AUC SCORE:  0.8615917699724519
AP：  0.8788838914504291
AUC SCORE:  0.8615810089531681
######################### Module cycle ： 180 ##########################
AP：  0.8788899392316641
AUC SCORE:  0.8615630739210285
AP：  0.8788974513138106
AUC SCORE:  0.

AP：  0.8821522818430907
AUC SCORE:  0.8655482380624426
######################### Module cycle ： 221 ##########################
AP：  0.8821392101905028
AUC SCORE:  0.8655051939853076
AP：  0.882218325450884
AUC SCORE:  0.8656486742424243
######################### Module cycle ： 222 ##########################
AP：  0.8822783124986464
AUC SCORE:  0.8656701962809917
AP：  0.8823044489093129
AUC SCORE:  0.8657240013774103
######################### Module cycle ： 223 ##########################
AP：  0.8823263866652618
AUC SCORE:  0.8656809573002754
AP：  0.8823425990156157
AUC SCORE:  0.8657562844352616
######################### Module cycle ： 224 ##########################
AP：  0.8823862062646901
AUC SCORE:  0.8657347623966942
AP：  0.8823522728753255
AUC SCORE:  0.8657706324609733
######################### Module cycle ： 225 ##########################
AP：  0.882280857118695
AUC SCORE:  0.8656737832874197
AP：  0.8822841834416333
AUC SCORE:  0.8656845443067034
######################### Module cycl

AP：  0.8839605245223338
AUC SCORE:  0.8684070821854912
AP：  0.883980813355247
AUC SCORE:  0.8684680612947657
######################### Module cycle ： 267 ##########################
AP：  0.8839714956292288
AUC SCORE:  0.8684429522497704
AP：  0.8839846714781743
AUC SCORE:  0.8685075183654729
######################### Module cycle ： 268 ##########################
AP：  0.8839568077296464
AUC SCORE:  0.8684752353076217
AP：  0.8839569683324293
AUC SCORE:  0.868503931359045
######################### Module cycle ： 269 ##########################
AP：  0.8839538592021458
AUC SCORE:  0.8684573002754821
AP：  0.8839451620572669
AUC SCORE:  0.8684429522497705
######################### Module cycle ： 270 ##########################
AP：  0.8839267137055908
AUC SCORE:  0.8683855601469237
AP：  0.8839041761628427
AUC SCORE:  0.8683425160697889
######################### Module cycle ： 271 ##########################
AP：  0.883860992872044
AUC SCORE:  0.8682384928833792
AP：  0.8839145710213867
AUC SCORE:  0.

In [6]:
print("################################  SUM UP #####################################")
print("AP FIRST AP MEAN: ", np.array(all_ap_first_ap).mean())
print("AP FIRST AP STD: ", np.array(all_ap_first_ap).std())

print("AP FIRST AUC MEAN: ", np.array(all_ap_first_auc).mean())
print("AP FIRST AUC STD: ", np.array(all_ap_first_auc).std())

print("AUC FIRST AP MEAN: ", np.array(all_auc_first_ap).mean())
print("AUC FIRST AP STD: ", np.array(all_auc_first_ap).std())

print("AUC FIRST AUC MEAN: ", np.array(all_auc_first_auc).mean())
print("AUC FIRST AUC STD: ", np.array(all_auc_first_auc).std())

################################  SUM UP #####################################
AP FIRST AP MEAN:  0.9042949406070706
AP FIRST AP STD:  0.013781105041689976
AP FIRST AUC MEAN:  0.8893596834825528
AP FIRST AUC STD:  0.014715774446795464
AUC FIRST AP MEAN:  0.9042728747517765
AUC FIRST AP STD:  0.013805001251950789
AUC FIRST AUC MEAN:  0.889394656795225
AUC FIRST AUC STD:  0.014676448148062307


In [7]:
filename = "Cora"    # The node index starts at 0, sep=None
nodes_number = 2708

embedding_dim = 24
n_clusters = 175
alpha = 1
beta = 4.2
DEC = 4.5
LR = 0.4

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []

for i in range(10):
    
    train_edge, test_edge, train_mask, test_mask = get_cora_graph(filename, nodes_number)
    
    ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)       
    raw_nodes_embedding = ini_embedding.weight.detach()        # Initial node embedding

    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(raw_nodes_embedding)
    cluster_centers = kmeans.cluster_centers_
    raw_cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids
    
    ap_first__ap = 0
    ap_first_auc = 0
    auc_first__ap = 0
    auc_first_auc = 0
    for module_epoch in range(300):
        print("######################### Module cycle ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
        raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
        if nodes_ap > ap_first__ap:
            ap_first__ap = nodes_ap
            ap_first_auc = nodes_auc
        if nodes_auc > auc_first_auc:
            auc_first_auc = nodes_auc
            auc_first__ap = nodes_ap
        raw_cluster_centers, cluster_ap, cluster_auc = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)
        if cluster_ap > ap_first__ap:
            ap_first__ap = cluster_ap
            ap_first_auc = cluster_auc
        if cluster_auc > auc_first_auc:
            auc_first_auc = cluster_auc
            auc_first__ap = cluster_ap
            
    all_ap_first_ap.append(ap_first__ap)
    all_ap_first_auc.append(ap_first_auc)
    all_auc_first_ap.append(auc_first__ap)
    all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
AP：  0.6185869704150155
AUC SCORE:  0.5878708964646464
AP：  0.6323847333306674
AUC SCORE:  0.6029327364554637
######################### Module cycle ： 1 ##########################
AP：  0.6475884402747298
AUC SCORE:  0.6173955463728191
AP：  0.6529794455937094
AUC SCORE:  0.6246915174471993
######################### Module cycle ： 2 ##########################
AP：  0.6591802568409936
AUC SCORE:  0.6330456554178145
AP：  0.6598742537615867
AUC SCORE:  0.6346598083103765
######################### Module cycle ： 3 ##########################
AP：  0.6614242712692162
AUC SCORE:  0.6392045454545454
AP：  0.6616690182773639
AUC SCORE:  0.640112058080808
######################### Module cycle ： 4 ##########################
AP：  0.6616558524890382
AUC SCORE:  0.6404169536271809
AP：  0.6627014464414037
AUC SCORE:  0.6418481691919191
######################### Module cycle ： 5 ##########################
AP：  0.6631630960546668
AUC SCO

AP：  0.8036338212841169
AUC SCORE:  0.7614497245179063
######################### Module cycle ： 46 ##########################
AP：  0.8052455384603211
AUC SCORE:  0.7630602904040404
AP：  0.8061010104509553
AUC SCORE:  0.7639857380624425
######################### Module cycle ： 47 ##########################
AP：  0.8078203550276108
AUC SCORE:  0.7659693526170799
AP：  0.8089780131605104
AUC SCORE:  0.7674256772268135
######################### Module cycle ： 48 ##########################
AP：  0.8102087602810597
AUC SCORE:  0.7691761363636364
AP：  0.8110116281128559
AUC SCORE:  0.7701087580348944
######################### Module cycle ： 49 ##########################
AP：  0.8124860265653624
AUC SCORE:  0.7719166092745637
AP：  0.8133455593547803
AUC SCORE:  0.7728169478879705
######################### Module cycle ： 50 ##########################
AP：  0.8146624685540598
AUC SCORE:  0.7745171889348026
AP：  0.8155873470614868
AUC SCORE:  0.775647095959596
######################### Module cycle ： 

AP：  0.8708614395935868
AUC SCORE:  0.8497044306703398
AP：  0.8710638169950202
AUC SCORE:  0.8501348714416898
######################### Module cycle ： 92 ##########################
AP：  0.8714342441833987
AUC SCORE:  0.8508271636822773
AP：  0.8717319729583171
AUC SCORE:  0.8513221705693297
######################### Module cycle ： 93 ##########################
AP：  0.8721656025638
AUC SCORE:  0.8520395718549127
AP：  0.8722862524859261
AUC SCORE:  0.8523767504591369
######################### Module cycle ： 94 ##########################
AP：  0.8726169092197548
AUC SCORE:  0.8530546946740128
AP：  0.8728264148626194
AUC SCORE:  0.8534923094582185
######################### Module cycle ： 95 ##########################
AP：  0.8732233389759249
AUC SCORE:  0.8541702536730946
AP：  0.8733751557677988
AUC SCORE:  0.8544859102387512
######################### Module cycle ： 96 ##########################
AP：  0.8728471005670009
AUC SCORE:  0.8551423324150598
AP：  0.8729039794039375
AUC SCORE:  0.85540

AP：  0.886199824615008
AUC SCORE:  0.8812485651974289
######################### Module cycle ： 137 ##########################
AP：  0.886290049543917
AUC SCORE:  0.8813705234159779
AP：  0.8864331746114206
AUC SCORE:  0.8815175906795225
######################### Module cycle ： 138 ##########################
AP：  0.8866061000428878
AUC SCORE:  0.8817615071166206
AP：  0.8867434473535462
AUC SCORE:  0.8819516184573004
######################### Module cycle ： 139 ##########################
AP：  0.8868462762537628
AUC SCORE:  0.8821058597337006
AP：  0.8869321744424508
AUC SCORE:  0.8822242309458219
######################### Module cycle ： 140 ##########################
AP：  0.887007528489004
AUC SCORE:  0.8823569501836548
AP：  0.8871220526630588
AUC SCORE:  0.8825291264921947
######################### Module cycle ： 141 ##########################
AP：  0.8872302355832504
AUC SCORE:  0.8826869547750229
AP：  0.8873609678256279
AUC SCORE:  0.8829093491735537
######################### Module cycle

AP：  0.8929698284964873
AUC SCORE:  0.892917097107438
AP：  0.8931208137386786
AUC SCORE:  0.8931107954545454
######################### Module cycle ： 183 ##########################
AP：  0.8931937816421567
AUC SCORE:  0.8932363406795225
AP：  0.8932924324486686
AUC SCORE:  0.8933869949494949
######################### Module cycle ： 184 ##########################
AP：  0.8934263595682708
AUC SCORE:  0.8935484102387512
AP：  0.8934729414244138
AUC SCORE:  0.8936380853994491
######################### Module cycle ： 185 ##########################
AP：  0.8935780998365962
AUC SCORE:  0.8938210227272727
AP：  0.8936971205682456
AUC SCORE:  0.8940111340679523
######################### Module cycle ： 186 ##########################
AP：  0.8937880231148781
AUC SCORE:  0.8941725493572085
AP：  0.8939133490104626
AUC SCORE:  0.8943949437557392
######################### Module cycle ： 187 ##########################
AP：  0.8939681899883518
AUC SCORE:  0.894484618916437
AP：  0.8940791323651713
AUC SCORE:  0

...

AP：  0.8763538763125472
AUC SCORE:  0.851436954775023
AP：  0.8767067595798377
AUC SCORE:  0.8520216368227731
######################### Module cycle ： 84 ##########################
AP：  0.8769410147223463
AUC SCORE:  0.8522942493112948
AP：  0.8772879818623491
AUC SCORE:  0.8529363234618916
######################### Module cycle ： 85 ##########################
AP：  0.8774451927237263
AUC SCORE:  0.8531658918732783
AP：  0.8778819392749282
AUC SCORE:  0.8538438360881543
######################### Module cycle ： 86 ##########################
AP：  0.8781405656542167
AUC SCORE:  0.8542957988980715
AP：  0.8785450995244648
AUC SCORE:  0.854873306932966
######################### Module cycle ： 87 ##########################
AP：  0.878752645615957
AUC SCORE:  0.8551566804407713
AP：  0.8791195472016652
AUC SCORE:  0.8557413624885215
######################### Module cycle ： 88 ##########################
AP：  0.8792998836205277
AUC SCORE:  0.8560516385445363
AP：  0.879749834484966
AUC SCORE:  0.856656

AP：  0.8966230178072156
AUC SCORE:  0.8787233126721763
######################### Module cycle ： 129 ##########################
AP：  0.896774106341241
AUC SCORE:  0.8787986398071626
AP：  0.8968575083874974
AUC SCORE:  0.8788667929292928
######################### Module cycle ： 130 ##########################
AP：  0.8970306609261881
AUC SCORE:  0.8790246212121211
AP：  0.8971005362836589
AUC SCORE:  0.8791501664370982
######################### Module cycle ： 131 ##########################
AP：  0.8972397816945765
AUC SCORE:  0.8792792986685032
AP：  0.8972859685597834
AUC SCORE:  0.8793546258034894
######################### Module cycle ： 132 ##########################
AP：  0.8974788142491823
AUC SCORE:  0.8795303891184574
AP：  0.8975289034475885
AUC SCORE:  0.8796093032598715
######################### Module cycle ： 133 ##########################
AP：  0.8976745136445246
AUC SCORE:  0.8796774563820018
AP：  0.8977809838717676
AUC SCORE:  0.8797599575298439
######################### Module cyc

AP：  0.9019100152324622
AUC SCORE:  0.8843584997704317
AP：  0.9019485646368843
AUC SCORE:  0.8844158918732783
######################### Module cycle ： 175 ##########################
AP：  0.9020093868777889
AUC SCORE:  0.8844517619375575
AP：  0.9020096178987448
AUC SCORE:  0.8844553489439854
######################### Module cycle ： 176 ##########################
AP：  0.9020764231714933
AUC SCORE:  0.8845199150596879
AP：  0.9020617988654428
AUC SCORE:  0.8844983930211203
######################### Module cycle ： 177 ##########################
AP：  0.9020944975963014
AUC SCORE:  0.8844948060146924
AP：  0.9021442981150543
AUC SCORE:  0.8845880681818182
######################### Module cycle ： 178 ##########################
AP：  0.9020728744888602
AUC SCORE:  0.8845199150596879
AP：  0.9021628670283257
AUC SCORE:  0.8846346992653811
######################### Module cycle ： 179 ##########################
AP：  0.9021893184171613
AUC SCORE:  0.8846490472910928
AP：  0.9022425891497138
AUC SCORE: 

AP：  0.9041357600550214
AUC SCORE:  0.8871778868227733
######################### Module cycle ： 220 ##########################
AP：  0.9041769828636099
AUC SCORE:  0.8872316919191918
AP：  0.9042080350865604
AUC SCORE:  0.8872962580348943
######################### Module cycle ： 221 ##########################
AP：  0.9042216023595547
AUC SCORE:  0.8873249540863177
AP：  0.9042552876103278
AUC SCORE:  0.8874253902662994
######################### Module cycle ： 222 ##########################
AP：  0.9042716295066022
AUC SCORE:  0.8874684343434344
AP：  0.9043341402168528
AUC SCORE:  0.887565283516988
######################### Module cycle ： 223 ##########################
AP：  0.9043560916154416
AUC SCORE:  0.8876477846648301
AP：  0.9044163500470355
AUC SCORE:  0.8877374598255281
######################### Module cycle ： 224 ##########################
AP：  0.9044275419360444
AUC SCORE:  0.8877661558769512
AP：  0.9044506145687278
AUC SCORE:  0.8878773530762167
######################### Module cyc

AP：  0.9071664205265386
AUC SCORE:  0.8919593663911844
AP：  0.9071665462462578
AUC SCORE:  0.8919916494490359
######################### Module cycle ： 266 ##########################
AP：  0.9072110179202434
AUC SCORE:  0.892066976584022
AP：  0.907243358947816
AUC SCORE:  0.8921028466483013
######################### Module cycle ： 267 ##########################
AP：  0.9072627578649307
AUC SCORE:  0.892110020661157
AP：  0.9073076312345358
AUC SCORE:  0.8921853477961433
######################### Module cycle ： 268 ##########################
AP：  0.9073417434892863
AUC SCORE:  0.8922140438475666
AP：  0.9073640905544773
AUC SCORE:  0.8922570879247016
######################### Module cycle ： 269 ##########################
AP：  0.9073706999402751
AUC SCORE:  0.8922642619375574
AP：  0.9073850102408544
AUC SCORE:  0.8923073060146923
######################### Module cycle ： 270 ##########################
AP：  0.9073733134244708
AUC SCORE:  0.8923180670339761
AP：  0.9073979659409479
AUC SCORE:  0.

In [8]:
print("################################  SUM UP #####################################")
print("AP FIRST AP MEAN: ", np.array(all_ap_first_ap).mean())
print("AP FIRST AP STD: ", np.array(all_ap_first_ap).std())

print("AP FIRST AUC MEAN: ", np.array(all_ap_first_auc).mean())
print("AP FIRST AUC STD: ", np.array(all_ap_first_auc).std())

print("AUC FIRST AP MEAN: ", np.array(all_auc_first_ap).mean())
print("AUC FIRST AP STD: ", np.array(all_auc_first_ap).std())

print("AUC FIRST AUC MEAN: ", np.array(all_auc_first_auc).mean())
print("AUC FIRST AUC STD: ", np.array(all_auc_first_auc).std())

################################  SUM UP #####################################
AP FIRST AP MEAN:  0.9020622610821357
AP FIRST AP STD:  0.009568123550287421
AP FIRST AUC MEAN:  0.8886765381083561
AP FIRST AUC STD:  0.01061783335096987
AUC FIRST AP MEAN:  0.9018280599978892
AUC FIRST AP STD:  0.009398913060874446
AUC FIRST AUC MEAN:  0.8891367510330579
AUC FIRST AUC STD:  0.010427244502697322
